<a href="https://colab.research.google.com/github/surajsrivathsa/image_registration/blob/main/Image_Pretraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble

In [ ]:
import numpy as np
import os
import sys
import torch
import nibabel as nb
import matplotlib.pyplot as plt
import glob

In [ ]:
import torch
from torch.nn import Sequential, Linear, ReLU
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn

import torch.utils.data as Data
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from math import log10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch-msssim
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM 

In [ ]:
!pip install pytorch-ignite
import ignite.metrics

In [ ]:
import torch.utils.data as data_utils

In [ ]:
data_path_hr = "/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/hrTest"
data_path_lr = "/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/usTest/Center6p25Mask/"

# data_path_t1 = "/content/drive/My Drive/Colab Notebooks/ImageRegistrationUsingDeepLearning/ICNet/Dataset/T1_resampled/"
# data_path_t2 = "/content/drive/My Drive/Colab Notebooks/ImageRegistrationUsingDeepLearning/ICNet/Dataset/T2_resampled/"
file_names_hr = sorted(glob.glob(os.path.join(data_path_hr)))
file_names_lr = sorted(glob.glob(os.path.join(data_path_lr)))

In [ ]:
!ls -ltr "/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/hrTest"

total 100
drwx------ 2 root root 4096 Jun  1 12:45 '136 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'
drwx------ 2 root root 4096 Jun  1 12:45 '134 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'
drwx------ 2 root root 4096 Jun  1 12:45 '132 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'
drwx------ 2 root root 4096 Jun  1 12:45 '130 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'
drwx------ 2 root root 4096 Jun  1 12:45 '128 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'
drwx------ 2 root root 4096 Jun  1 12:45 '126 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'
drwx------ 2 root root 4096 Jun  1 12:45 '124 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'
drwx------ 2 root root 4096 Jun  1 12:45 '122 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'
drwx------ 2 root root 4096 Jun  1 12:45 '120 - t1_fl3d_tra_dyn_noPAT_noPP_noO

In [ ]:
'134 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm.nii.gz'.split(".")[0]

'134 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm'

In [ ]:
# Create file list
file_names_lr = []
file_names_hr = []
for path, directories, files in os.walk(data_path_hr):
  for fl in files:
    if fl.endswith("nii.gz"):
      file_names_hr.append(os.path.join(path,fl ))

for path, directories, files in os.walk(data_path_lr):
  for fl in files:
    if fl.endswith("nii.gz"):
      file_names_lr.append(os.path.join(path,fl ))

# Sort the file names so we have corresponding files for high and low res at same indices
file_names_lr = sorted(file_names_lr)
file_names_hr = sorted(file_names_hr)

In [ ]:
file_names_hr[:2]

['/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/hrTest/118 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm/118 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm.nii.gz',
 '/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/hrTest/120 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm/120 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm.nii.gz']

In [ ]:
file_names_lr[:2]

['/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/usTest/Center6p25Mask/118 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm/118 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm.nii.gz',
 '/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/usTest/Center6p25Mask/120 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm/120 - t1_fl3d_tra_dyn_noPAT_noPP_noOpt_fatSat_25_7p6s_imgFilShrp5ed1sm.nii.gz']

# Create dataset

In [ ]:
def load_4D(name):
    
    
    X_nb = nb.load(name)
    X_np = X_nb.dataobj

    # We pad 10 and 10 along z direction to make it to 64 from 44. 64 is in powers of 2 which is similar to 256 and 192, Hence convolution becomes easier
    
    resamplng_shape = (256, 192, 64)
    lowest_intensity_in_image = np.min(X_np)
    model_np = np.full(shape=resamplng_shape, fill_value=lowest_intensity_in_image)
    x_dim, y_dim, z_dim = X_np.shape
    z_ltail = (resamplng_shape[2] - z_dim)//2
    z_rtail = resamplng_shape[2] - z_ltail
    model_np[:, :, z_ltail:z_rtail] = X_np[:, :, :]

    # Uncomment the below line if we wnat to add 4th dimension, might be useful for time/temporal concatenation
    # model_np = np.reshape(model_np, (1,)+ model_np.shape)
    return model_np

def imgnorm(N_I,index1=0.0001,index2=0.0001):
    I_sort = np.sort(N_I.flatten())
    I_min = I_sort[int(index1*len(I_sort))]
    I_max = I_sort[-int(index2*len(I_sort))]
    N_I =1.0*(N_I-I_min)/(I_max-I_min)
    N_I[N_I>1.0]=1.0
    N_I[N_I<0.0]=0.0
    N_I2 = N_I.astype(np.float32)
    return N_I2

def Norm_Zscore(img):
    img= (img-np.mean(img))/np.std(img) 
    return img

def load_3D(name):
  X_nb = nb.load(name)
  X_np = X_nb.dataobj
  return X_np

In [ ]:
myimg = imgnorm(load_4D(file_names_hr[0]))
print("Before shape: {} and After shape: {} ".format(load_3D(file_names_hr[0]).shape, myimg.shape))
print("Before min: {} and After min: {} ".format(  np.min(load_3D(file_names_hr[0])), np.min(imgnorm(myimg)) ))
print("Before max: {} and After max: {} ".format(  np.max(load_3D(file_names_hr[0])), np.max(imgnorm(myimg)) ))


Before shape: (256, 192, 44) and After shape: (256, 192, 64) 
Before min: 0 and After min: 0.0 
Before max: 286 and After max: 1.0 


In [ ]:
# Run this function only when you dont have lowres_patches.pt and highres_patches.pt or you need to recreate it

def create_3d_patches(lr_filenames, hr_filenames):
  # Initialize  empty tensors of that can hold patches of size 64, 48, 16 for all files combines

  hr_all_img_patches = torch.empty(size=(len(file_names_lr) * 64, 1, 64, 48, 16))
  lr_all_img_patches = torch.empty(size=(len(file_names_lr) * 64, 1, 64, 48, 16))
  ind = 0
  for hr_fl, lr_fl in zip(hr_filenames, lr_filenames):

    # Normalize each image inyensiy betwen 0 and 1, and also concatenate padding of 20 for z dimension to make it from 44 to 64
    hr_img_tnsr = torch.from_numpy(imgnorm(load_4D(hr_fl)))

    # Create patches
    hr_patches = hr_img_tnsr.unfold(2, 16, 16).unfold(1, 48, 48).unfold(0, 64, 64)
    hr_patches = hr_patches.contiguous().view(-1, 64, 48, 16)
    hr_all_img_patches[ind: ind+64, 0, :, :, :] = hr_patches

    # hr_patches = hr_img_tnsr.unfold(2, 8, 8).unfold(1, 24, 24).unfold(0, 32, 32)
    # hr_patches = hr_patches.contiguous().view(-1, 32, 24, 8)

    lr_img_tnsr = torch.from_numpy(imgnorm(load_4D(lr_fl)))
    lr_patches = lr_img_tnsr.unfold(2, 16, 16).unfold(1, 48, 48).unfold(0, 64, 64)
    lr_patches = lr_patches.contiguous().view(-1, 64, 48, 16)
    lr_all_img_patches[ind: ind+64, 0, :, :, :] = lr_patches

    # lr_patches = lr_img_tnsr.unfold(2, 8, 8).unfold(1, 24, 24).unfold(0, 32, 32)
    # lr_patches = lr_patches.contiguous().view(-1, 32, 24, 8)

    ind = ind+1
  
  print("Processed {} files".format(ind))
  return lr_all_img_patches, lr_all_img_patches


In [ ]:
lrpatches , hrpatches = create_3d_patches(file_names_lr, file_names_hr )

Processed 25 files


In [ ]:
dataset = data_utils.TensorDataset(lrpatches, hrpatches)

In [ ]:
torch.save(lrpatches, "/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/lowres_patches.pth")
torch.save(hrpatches, "/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/highres_patches.pth")
torch.save(dataset , "/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/patch_dataset.pth")

In [ ]:
patch_dataset = torch.load("/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/patch_dataset.pth")

In [ ]:
patch_data_loader = DataLoader(patch_dataset,batch_size=16, shuffle=True, drop_last=True)

In [ ]:
ind = 0
for lrpatches, hrpatches in patch_data_loader:
  
  print(lrpatches.shape, hrpatches.shape)
  print(lrpatches.min(), hrpatches.min())
  print(lrpatches.max(), hrpatches.max())
  print(lrpatches.mean(), hrpatches.mean())
  print("============================")
  if(ind > 2):
    break;
  ind = ind+1

torch.Size([16, 1, 64, 48, 16]) torch.Size([16, 1, 64, 48, 16])
tensor(0.) tensor(0.)
tensor(0.8717) tensor(0.8717)
tensor(0.0168) tensor(0.0168)
torch.Size([16, 1, 64, 48, 16]) torch.Size([16, 1, 64, 48, 16])
tensor(0.) tensor(0.)
tensor(0.) tensor(0.)
tensor(0.) tensor(0.)
torch.Size([16, 1, 64, 48, 16]) torch.Size([16, 1, 64, 48, 16])
tensor(0.) tensor(0.)
tensor(0.8064) tensor(0.8064)
tensor(0.0016) tensor(0.0016)
torch.Size([16, 1, 64, 48, 16]) torch.Size([16, 1, 64, 48, 16])
tensor(0.) tensor(0.)
tensor(0.8808) tensor(0.8808)
tensor(0.0150) tensor(0.0150)


# Miscellaneous

In [ ]:
te = torch.empty(size=(len(file_names_lr) * 512, 32, 24, 8))
te.shape

torch.Size([12800, 32, 24, 8])

In [ ]:
mytnsr = torch.randn(size=(256, 192, 64))

In [ ]:
imgtnsr = torch.from_numpy(myimg)

In [ ]:
t1_patches = mytnsr.unfold(2, 16, 16).unfold(1, 48, 48).unfold(0, 64, 64)
t1_patches = t1_patches.contiguous().view(-1, 64, 48, 16)

In [ ]:
t1_patches[33:, :, :].mean()

tensor(-0.0002)

In [ ]:
t1_patches = imgtnsr[0, :, :, :].unfold(2, 8, 8).unfold(1, 24, 24).unfold(0, 32, 32)
t1_patches = t1_patches.contiguous().view(-1, 32, 24, 8)

In [ ]:
t1_patches.shape

torch.Size([512, 32, 24, 8])

In [ ]:
t1_patches[3:, :, :].mean()

tensor(0.1165)

# Model Creation

In [ ]:
class VSRNet(nn.Module):
  def __init__(self,in_channel=1 ):
        self.in_channel = in_channel
        super(VSRNet, self).__init__()
        self.layer1 = self.conv_layer(self.in_channel, 64, kernel_size=9, stride=1, padding=5, bias=True)
        self.layer2 = self.conv_layer(64, 32, kernel_size=7, stride=1, padding=3, bias=True)
        self.layer3 = self.conv_layer(32, 8, kernel_size=5, stride=1, padding=2, bias=True)
        self.layer4 = self.conv_layer(8, 1, kernel_size=5, stride=1, padding=1, bias=True)

  def conv_layer(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
    layer = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=bias),
                nn.LeakyReLU())
    return layer


  def forward(self, x):
          
        encoded_output = self.layer1(x)
        encoded_output = self.layer2(encoded_output)
        encoded_output = self.layer3(encoded_output)
        encoded_output = self.layer4(encoded_output)

        return encoded_output

In [ ]:
vsrmodel = VSRNet()

In [ ]:
myop = vsrmodel(torch.randn((4,1,64,48,16)))

In [ ]:
myop.shape

torch.Size([4, 1, 64, 48, 16])

# Losses

In [ ]:
ssimloss = SSIM(data_range=1.0, channel=1, spatial_dims=3)

In [ ]:
ssimloss( torch.ones((4,1,64, 48, 16)), torch.zeros((4,1,64, 48, 16)) )

tensor(9.9990e-05)

In [ ]:
mseloss = nn.MSELoss()

In [ ]:
mse = mseloss(torch.ones((4,1,64, 48, 16)), torch.zeros((4,1,64, 48, 16)) )
psnrloss = 10 * log10(1 / mse.item())
print(mse, psnrloss)

In [ ]:
mse = mseloss(torch.ones((4,1,64, 48, 16)), torch.zeros((4,1,64, 48, 16)) )
psnrloss = 10 * log10(1 / mse.item())
print(mse, psnrloss)

In [ ]:
mse = mseloss(torch.randn((4,1,64, 48, 16)), torch.randn((4,1,64, 48, 16)) )
psnrloss = 10 * log10(1 / mse.item())
print(mse, psnrloss)

tensor(2.0016) -3.0137859148779738


# Training

In [ ]:
training_generator = DataLoader(patch_dataset,batch_size=64, shuffle=True, drop_last=True)

In [ ]:
del vsrnet_model

In [ ]:
vsrnet_model = VSRNet().to("cuda") # assigning cuda to model

In [ ]:
lr = 1e-4
mse_loss_fnc = nn.MSELoss()
ssim_loss_fnc = SSIM(data_range=1.0, channel=1, spatial_dims=3)

In [ ]:
optimizer = torch.optim.Adam(vsrnet_model.parameters(), lr=lr ) 
model_dir = '/content/drive/MyDrive/Colab Notebooks/MRI_Project/model'

if not os.path.isdir(model_dir):
  os.mkdir(model_dir)
step=0

In [ ]:
def one_epoch_run(epoch=1):
  batch_number = 0
  mse_loss_lst = []
  ssim_loss_lst = []
  psnr_loss_lst = []
  total_loss_lst = []
  for X ,Y in training_generator:

    X = X.cuda().float()
    Y = Y.cuda().float()

    Y_pred = vsrnet_model(X) 
    mse_loss = mse_loss_fnc(Y, Y_pred)
    ssim_loss = ssim_loss_fnc(Y, Y_pred)
    psnr_loss = torch.Tensor([10 * log10(1 / mse_loss.item())]).to("cuda")

    # PSNR is ranges from 20DB to 100DB, Hence multiply it with 0.01 to bring in sae range as [0,1]
    # For SSiM and PSNR, More the value the better the quality, but for mse its the opposite, Hence for these tqo losses normalize them between [0,1] and subtract by 1
    total_loss = 1.0 * mse_loss + 1.0 * (1.0 - ssim_loss) + 1.0 * (1.0 - psnr_loss*0.01)
  
    optimizer.zero_grad()          
    total_loss.backward()
    optimizer.step()

    total_loss_lst.append(total_loss.detach().cpu().numpy().item())
    mse_loss_lst.append(mse_loss.detach().cpu().numpy().item())
    ssim_loss_lst.append(ssim_loss.detach().cpu().numpy().item())
    psnr_loss_lst.append(psnr_loss.detach().cpu().numpy().item())

    del X, Y, Y_pred

    batch_number = batch_number + 1
    # print("Batch: {}".format(batch_number))

  if (epoch%10 == 0):
    modelname = model_dir + '/' + "pretrained_superresolution" + str(epoch+0) + '.pth'
    torch.save({"vsrnet_model": vsrnet_model.state_dict()}, modelname)
    print("epoch: {}".format(epoch+0))
    print("Average loss per epoch: {}, {}, {}".format( sum(mse_loss_lst)/len(mse_loss_lst), sum(ssim_loss_lst)/len(ssim_loss_lst), sum(psnr_loss_lst)/len(psnr_loss_lst)  ))
    print("======= =============== ===========")
    print("Saving model checkpoints")
    print("======= =============== ===========")
    print()
  elif (epoch%2 == 0):
    print("epoch: {}".format(epoch+0))
    print("Average loss per epoch: {}, {}, {}".format( sum(mse_loss_lst)/len(mse_loss_lst), sum(ssim_loss_lst)/len(ssim_loss_lst), sum(psnr_loss_lst)/len(psnr_loss_lst)  ))
    print("======= =============== ===========")
    print()

  return mse_loss_lst, ssim_loss_lst, psnr_loss_lst, total_loss_lst


In [ ]:
epochs = 100
for e in range(epochs):
  mse_loss_lst, ssim_loss_lst, psnr_loss_lst, total_loss_lst = one_epoch_run(epoch=e)

epoch: 0
Average loss per epoch: 0.0009743702379637398, 0.9282254993915557, 32.49748680114746
======= =============== ===========
Saving model checkpoints
======= =============== ===========

epoch: 2
Average loss per epoch: 0.0002274121378286509, 0.9977713990211486, 37.911679077148435
======= =============== ===========

epoch: 4
Average loss per epoch: 0.00018288389030203688, 0.9990405011177063, 38.516942749023436
======= =============== ===========

epoch: 6
Average loss per epoch: 0.00014014900873007718, 0.9995319604873657, 39.7111067199707
======= =============== ===========

epoch: 8
Average loss per epoch: 0.00013782075759081637, 0.9992779183387757, 40.4745735168457
======= =============== ===========

epoch: 10
Average loss per epoch: 0.00014586778663215228, 0.9993899011611939, 39.9764860534668
======= =============== ===========
Saving model checkpoints
======= =============== ===========

epoch: 12
Average loss per epoch: 0.00011505705944728106, 0.9997169327735901, 40.7160260

# Validation

In [ ]:
validation_generator = DataLoader(patch_dataset,batch_size=4, shuffle=True, drop_last=True)

In [ ]:
output_pred_folder = "/content/drive/MyDrive/Colab Notebooks/MRI_Project/sr/datasets/MRI_DATASET/Output_Predictions"

In [ ]:
# Addd deformable part
vsrnet_model_inference = VSRNet().to("cuda") # assigning cuda to model

In [ ]:
checkpoint = torch.load(model_dir + '/' + "pretrained_superresolution" + str(0) + '.pth')
vsrnet_model_inference.load_state_dict(checkpoint["vsrnet_model"])
vsrnet_model_inference.eval()

VSRNet(
  (layer1): Sequential(
    (0): Conv3d(1, 64, kernel_size=(9, 9, 9), stride=(1, 1, 1), padding=(5, 5, 5))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer2): Sequential(
    (0): Conv3d(64, 32, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer3): Sequential(
    (0): Conv3d(32, 8, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer4): Sequential(
    (0): Conv3d(8, 1, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
)

In [ ]:
counter = 0

In [ ]:
def one_epoch_full_model_inference(epoch=1, batch_size=4, counter=0):
  example_number = 0

  for X,Y in validation_generator:

    X = X.float().to("cuda")
    Y = Y.float().to("cuda")

    Y_pred = vsrnet_model_inference(X) 

    Y_pred_tensor = Y_pred.detach().cpu()

    Y_pred_np = Y_pred_tensor.numpy()

    Y_pred_nb = nb.Nifti1Image(Y_pred_np[0,0,:,:,:], np.eye(4)) 

    nb.save(Y_pred_nb, os.path.join(output_pred_folder , 'predicted_nb_' + str(counter) + '.nii.gz'))
    del X, Y, Y_pred

    counter = counter + 1

    if(counter > 5):
      break;

In [ ]:
one_epoch_full_model_inference()